<a href="https://colab.research.google.com/github/markaaronslater/NMT/blob/master/playground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%load_ext autoreload
%autoreload 2

from google.colab import drive
drive.mount('/content/gdrive')

!pip install subword-nmt # for segmenting words into subwords
!pip install stanza # for tokenizing corpus and tagging with morphological data
!pip install sacremoses # for detokenizing model predictions

# make sure using GPU
# (Runtime -> Change runtime type -> Hardware accelerator = GPU).
!nvidia-smi

# recommended: place cloned NMT folder in Google drive folder 'My Drive':
path = '/content/gdrive/My Drive/NMT/'
corpus_path = path + 'corpuses/iwslt16_en_de/'
config_path = path + 'configs/'
# give model a name representing, e.g., major hyperparameter setting differences from other models, etc.
model_name = 'old_replica/' # name of model tensor batches, hyperparameters, etc., saved as pickle file inside data_path
checkpoint_path = path + 'checkpoints/' + model_name

### ??how to give them access to my pre-trained model?? too large to commit to github??






The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
     |████████████████████████████████| 235kB 2.8MB/s 
     |████████████████████████████████| 890kB 2.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=bba08adb6ab912c8c9622af3db3a4e6d86bad59a0aaf136093e182c28287048a
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [ ]:
# pre-trained model employs a subword-level vocabulary
p = corpus_path + 'subword_segmented/' # point p to the preprocessed corpuses to be directly used by model 
